# Prediction of Early-Stage Melanoma Recurrence Using Clinical and Histopathologic Features

## classifcation tasks

Guihong Wan, July/27/2022   
Massachusetts General Hospital, Harvard Medical School

In [2]:
run 'header.py'

In [3]:
run 'Utils.py'

## 1. Data preprocessing

In [4]:
# 30 artificial samples created to demonstrate the running of the codes.
melanoma_data = pd.read_csv("data/melanoma_example_v1.0.csv")

In [5]:
print(melanoma_data.shape)
print(melanoma_data['Recurrence'].value_counts(), "\n")

(30, 133)
0    20
1    10
Name: Recurrence, dtype: int64 



In [6]:
melanoma_data = melanoma_data.drop(columns = ['MelanomaID']+nomi_features)
melanoma_data.tail()

,Recurrence,Dia2Recur,Site,TrdxAgeAtDx,medianincome,CCItotal_psuedoMedian,cmhistory,fthickness,fAnatomicLevel,fMitoses,...,CAID_binary_Unavailable,CAID_binary_Yes,SAID_binary_No,SAID_binary_Unavailable,SAID_binary_Yes,newNODE_0.0,newNODE_1.0,newNODE_2.0,newNODE_3.0,newNODE_4.0
25,1,1.7,MGB,87,82.416,4,0,5.0,6,4,...,1,0,0,1,0,0,0,1,0,0
26,1,2.0,MGB,86,82.416,4,0,5.0,4,14,...,1,0,0,1,0,0,0,1,0,0
27,1,2.5,MGB,69,126.081,2,1,20.0,5,14,...,1,0,0,1,0,0,0,0,0,1
28,1,2.2,MGB,32,126.081,0,0,1.9,4,14,...,0,0,1,0,0,0,0,0,0,1
29,1,3.0,MGB,70,126.081,2,0,3.0,4,14,...,1,0,0,1,0,0,0,0,0,1


### MGB:DFCI split

In [24]:
data_MGH = melanoma_data[melanoma_data['Site'] == "MGB"]
data_DFCI = melanoma_data[melanoma_data['Site'] == "DFCI"]

print(data_MGH['Recurrence'].value_counts(), "\n")
print(data_DFCI['Recurrence'].value_counts(), "\n")

X_mgh = data_MGH.drop(columns = ['Recurrence', 'Site', 'Dia2Recur'])
y_mgh = data_MGH['Recurrence']

X_test = data_DFCI.drop(columns = ['Recurrence', 'Site', 'Dia2Recur'])
y_test = data_DFCI['Recurrence']

0    10
1     5
Name: Recurrence, dtype: int64 

0    10
1     5
Name: Recurrence, dtype: int64 



## 2. Prediction performance with demographics, medical comorbidities and tumor features

In [8]:
DEMOGRAPHCIS = False  # demographics only
MEDICAL = False  # demographics + meidcal history only
ALL = True # all features
assert sum([DEMOGRAPHCIS, MEDICAL, ALL]) == 1

In [9]:
if DEMOGRAPHCIS:
    combo = []
    for col in list(melanoma_data.columns):
        for target in demographic_features:
            if target in col:
                combo.append(col)

if MEDICAL:
    combo = []
    for col in list(melanoma_data.columns):
        for target in demographic_features + medical_features:
            if target in col:
                combo.append(col)
if ALL:
    combo = X_mgh.columns

print(combo)

Index(['TrdxAgeAtDx', 'medianincome', 'CCItotal_psuedoMedian', 'cmhistory',
       'fthickness', 'fAnatomicLevel', 'fMitoses', 'totalmargins',
       'GenderNm_Male', 'Race_White',
       ...
       'CAID_binary_Unavailable', 'CAID_binary_Yes', 'SAID_binary_No',
       'SAID_binary_Unavailable', 'SAID_binary_Yes', 'newNODE_0.0',
       'newNODE_1.0', 'newNODE_2.0', 'newNODE_3.0', 'newNODE_4.0'],
      dtype='object', length=101)


In [10]:
X_mgh = data_MGH.drop(columns = ['Recurrence', 'Site'])
y_mgh = data_MGH['Recurrence']


X_dfci = data_DFCI.drop(columns = ['Recurrence', 'Site'])
y_dfci = data_DFCI['Recurrence']

X_mgh = X_mgh[combo]
X_dfci = X_dfci[combo]

In [14]:
scoring = {'accuracy': 'accuracy',
           'roc_auc': 'roc_auc',
           'recall': 'recall',
           'precision': 'precision',
           'specifty': make_scorer(get_specificity),
           'tp': make_scorer(get_TP),
           'fp': make_scorer(get_FP),
          }
tuning_scoring = "roc_auc"

name = ''
nFold = 5 # five cross validation
nTrials = 50

### GradientBoostingClassifier

In [15]:
rets_internal = {'acc':[], 'auc': [], 'prec': [], 'recall':[], 'sp': []}
rets_cross = {'acc':[], 'auc': [], 'prec': [], 'recall':[], 'sp': []}

for i in range(nTrials):
    
    if i%10==0: LOG=True
    else: LOG = False
        
    # 1. down sampling, then the number of recurrences == the number of non-recurrences
    X, y = RandomUnderSampler().fit_resample(X_mgh, y_mgh)
    X_test, y_test = RandomUnderSampler().fit_resample(X_dfci, y_dfci)


    X = StandardScaler().fit_transform(X)
    X_test = StandardScaler().fit_transform(X_test)
    
    
    if i==0: print(y.value_counts())
    if i==0: print(y_test.value_counts())
    
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    
    # 2. CV to tune the parameter
    param_grid = {'learning_rate':[0.01, 0.02, 0.03, 0.04, 0.05],
                 'min_samples_leaf':[1,2],
                 'max_depth':[1,2],
                 'n_estimators':[100]}
    CLF = GradientBoostingClassifier(validation_fraction=0.0001)
    gsearch = GridSearchCV(estimator = CLF, 
                           param_grid = param_grid, 
                           scoring=tuning_scoring , cv=cv)
    gsearch.fit(X,y)
    if LOG: print(gsearch.best_params_)
    
    # 3. internal validation
    model = GradientBoostingClassifier(n_estimators=gsearch.best_params_['n_estimators'],
                                       validation_fraction = 0.0001,
                                       learning_rate=gsearch.best_params_['learning_rate'],
                                       min_samples_leaf=gsearch.best_params_['min_samples_leaf'],
                                       max_depth=gsearch.best_params_['max_depth'],
                                      )
    scores = cross_validate(model, X, y, 
                            scoring=scoring, 
                            cv=cv, 
                            return_train_score= True)
    
    if LOG:
        print(i,'auc',scores['train_roc_auc'].mean(),scores['test_roc_auc'].mean())
        print(i,'ppv',scores['train_precision'].mean(),scores['test_precision'].mean())
        
    rets_internal['acc']  += list(scores['test_accuracy'])
    rets_internal['auc']  += list(scores['test_roc_auc'])
    rets_internal['prec']  += list(scores['test_precision'])
    rets_internal['recall']  += list(scores['test_recall'])
    rets_internal['sp']  += list(scores['test_specifty'])
    
    # 4. external validation
    model = GradientBoostingClassifier(n_estimators=gsearch.best_params_['n_estimators'],
                                       validation_fraction = 0.0001,
                                       learning_rate=gsearch.best_params_['learning_rate'],
                                       min_samples_leaf=gsearch.best_params_['min_samples_leaf'],
                                       max_depth=gsearch.best_params_['max_depth'],
                                      )
    
    _, acc, auc, precision, recall, sp, tp, fp = baseline_fn(model, X, X_test, y, y_test, name = '')
    rets_cross['acc']  += [acc]
    rets_cross['auc']  += [auc]
    rets_cross['prec']  += [precision]
    rets_cross['recall']  += [recall]
    rets_cross['sp']  += [sp]
    if LOG:
        print('external','auc',auc,'ppv',precision) 
    

print()
ret = getmeanAndCI(rets_internal['auc'], "AUC")
ret = getmeanAndCI(rets_internal['prec'], "PPV")
ret = getmeanAndCI(rets_internal['recall'], "Sensitivity")
ret = getmeanAndCI(rets_internal['sp'], "Specificity")
ret = getmeanAndCI(rets_internal['acc'], "ACC")
print()
ret = getmeanAndCI(rets_cross['auc'], "AUC")
ret = getmeanAndCI(rets_cross['prec'], "PPV")
ret = getmeanAndCI(rets_cross['recall'], "Sensitivity")
ret = getmeanAndCI(rets_cross['sp'], "Specificity")
ret = getmeanAndCI(rets_cross['acc'], "ACC")

0    5
1    5
Name: Recurrence, dtype: int64
0    5
1    5
Name: Recurrence, dtype: int64
{'learning_rate': 0.01, 'max_depth': 1, 'min_samples_leaf': 1, 'n_estimators': 100}
0 auc 1.0 0.7
0 ppv 1.0 0.5
external auc 0.6000000000000001 ppv 0.6
{'learning_rate': 0.01, 'max_depth': 1, 'min_samples_leaf': 1, 'n_estimators': 100}
10 auc 1.0 0.8
10 ppv 1.0 0.4
external auc 0.6399999999999999 ppv 1.0

AUC:0.715
 CI:0.634-0.796
PPV:0.495
 CI:0.409-0.581
Sensitivity:0.62
 CI:0.523-0.717
Specificity:0.58
 CI:0.482-0.678
ACC:0.6
 CI:0.529-0.671

AUC:0.553
 CI:0.472-0.634
PPV:0.563
 CI:0.454-0.671
Sensitivity:0.54
 CI:0.487-0.593
Specificity:0.48
 CI:0.324-0.636
ACC:0.51
 CI:0.418-0.602


In [17]:
if ALL:
    rets_internal_GB = rets_internal
    rets_cross_GB = rets_cross

if DEMOGRAPHCIS:
    rets_internal_GB_demo = rets_internal
    rets_cross_GB_demo = rets_cross

if MEDICAL:
    rets_internal_GB_med = rets_internal  
    rets_cross_GB_med = rets_cross

In [26]:
_,p = st.ttest_ind(rets_internal_GB['auc'], rets_cross_GB['auc'])
print("auc %.3f" % p)
_,p = st.ttest_ind(rets_internal_GB['prec'], rets_cross_GB['prec'])
print("ppv %.3f" % p)
_,p = st.ttest_ind(rets_internal_GB['recall'], rets_cross_GB['recall'])
print("sn %.3f" % p)
_,p = st.ttest_ind(rets_internal_GB['sp'], rets_cross_GB['sp'])
print("sp %.3f" % p)
_,p = st.ttest_ind(rets_internal_GB['acc'], rets_cross_GB['acc'])
print("acc %.3f" % p)

auc 0.086
ppv 0.501
sn 0.469
sp 0.390
acc 0.275


### LogisticRegression

In [19]:
rets_internal = {'acc':[], 'auc': [], 'prec': [], 'recall':[], 'sp': []}
rets_cross = {'acc':[], 'auc': [], 'prec': [], 'recall':[], 'sp': []}
for i in range(nTrials):
    if i%10==0: LOG=True
    else: LOG = False
        
    # 1. down sampling
    X, y = RandomUnderSampler().fit_resample(X_mgh, y_mgh)
    X_test, y_test = RandomUnderSampler().fit_resample(X_dfci, y_dfci)
    
    X = StandardScaler().fit_transform(X)
    X_test = StandardScaler().fit_transform(X_test)
    
    if i==0: print(y.value_counts())
    if i==0: print(y_test.value_counts())
    
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    
    # 2. CV to tune the parameter
    param_grid = {'C':[0.001, 0.002, 0.003,0.004, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03,0.035, 0.04, 0.05, 
                       0.1,0.2,0.3],
                 'penalty':['l2']}
    CLF = LogisticRegression(solver= 'liblinear')
    gsearch = GridSearchCV(estimator = CLF, 
                            param_grid = param_grid, 
                            scoring=tuning_scoring , cv=cv)
    gsearch.fit(X,y)
    if LOG: print(gsearch.best_params_)
    
    
    # 3. internal validation
    model = LogisticRegression(solver= 'liblinear',
                               penalty= gsearch.best_params_['penalty'], 
                               C=gsearch.best_params_['C'] )
    scores = cross_validate(model, X, y, 
                            scoring=scoring, 
                            cv=cv, return_train_score= True)
    rets_internal['acc']  += list(scores['test_accuracy'])
    rets_internal['auc']  += list(scores['test_roc_auc'])
    rets_internal['prec']  += list(scores['test_precision'])
    rets_internal['recall']  += list(scores['test_recall'])
    rets_internal['sp']  += list(scores['test_specifty'])
    
    if LOG:
        print('auc',scores['train_roc_auc'].mean(),scores['test_roc_auc'].mean())
        print('ppv',scores['train_precision'].mean(),scores['test_precision'].mean())
    
    # 4. external validation
    model = LogisticRegression(solver= 'liblinear',
                               penalty= gsearch.best_params_['penalty'], 
                               C=gsearch.best_params_['C'] )
    
    _, acc, auc, precision, recall, sp, tp, fp = baseline_fn(
        model, X, X_test, y, y_test, name = '')
    
    rets_cross['acc']  += [acc]
    rets_cross['auc']  += [auc]
    rets_cross['prec']  += [precision]
    rets_cross['recall']  += [recall]
    rets_cross['sp']  += [sp]

print()
ret = getmeanAndCI(rets_internal['auc'], "AUC")
ret = getmeanAndCI(rets_internal['prec'], "PPV")
ret = getmeanAndCI(rets_internal['recall'], "Sensitivity")
ret = getmeanAndCI(rets_internal['sp'], "Specificity")
ret = getmeanAndCI(rets_internal['acc'], "ACC")
print()
ret = getmeanAndCI(rets_cross['auc'], "AUC")
ret = getmeanAndCI(rets_cross['prec'], "PPV")
ret = getmeanAndCI(rets_cross['recall'], "Sensitivity")
ret = getmeanAndCI(rets_cross['sp'], "Specificity")
ret = getmeanAndCI(rets_cross['acc'], "ACC")

0    5
1    5
Name: Recurrence, dtype: int64
0    5
1    5
Name: Recurrence, dtype: int64
{'C': 0.001, 'penalty': 'l2'}
auc 0.975 0.4
ppv 0.95 0.3
{'C': 0.001, 'penalty': 'l2'}
auc 1.0 0.4
ppv 1.0 0.1

AUC:0.64
 CI:0.544-0.736
PPV:0.55
 CI:0.463-0.637
Sensitivity:0.66
 CI:0.566-0.754
Specificity:0.68
 CI:0.587-0.773
ACC:0.67
 CI:0.605-0.735

AUC:0.424
 CI:0.36-0.488
PPV:0.455
 CI:0.396-0.514
Sensitivity:0.44
 CI:0.351-0.529
Specificity:0.49
 CI:0.419-0.561
ACC:0.465
 CI:0.408-0.522


In [21]:
if ALL:
    rets_internal_LR = rets_internal
    rets_cross_LR = rets_cross

if DEMOGRAPHCIS:
    rets_internal_LR_demo = rets_internal
    rets_cross_LR_demo = rets_cross

if MEDICAL:
    rets_internal_LR_med = rets_internal 
    rets_cross_LR_med = rets_cross

In [27]:
_,p = st.ttest_ind(rets_internal_LR['auc'], rets_cross_LR['auc'])
print("auc %.3f" % p)
_,p = st.ttest_ind(rets_internal_LR['prec'], rets_cross_LR['prec'])
print("ppv %.3f" % p)
_,p = st.ttest_ind(rets_internal_LR['recall'], rets_cross_LR['recall'])
print("sn %.3f" % p)
_,p = st.ttest_ind(rets_internal_LR['sp'], rets_cross_LR['sp'])
print("sp %.3f" % p)
_,p = st.ttest_ind(rets_internal_LR['acc'], rets_cross_LR['acc'])
print("acc %.3f" % p)

auc 0.050
ppv 0.344
sn 0.045
sp 0.076
acc 0.007


### MLP Classifier

In [23]:
rets_internal = {'acc':[], 'auc': [], 'prec': [], 'recall':[], 'sp': []}
rets_cross = {'acc':[], 'auc': [], 'prec': [], 'recall':[], 'sp': []}

for i in range(nTrials):
    if i%10==0: LOG=True
    else: LOG = False
        
    # 1. down sampling
    X, y = RandomUnderSampler().fit_resample(X_mgh, y_mgh)
    X_test, y_test = RandomUnderSampler().fit_resample(X_dfci, y_dfci)
    
    X = StandardScaler().fit_transform(X)
    X_test = StandardScaler().fit_transform(X_test)
    
    if i==0: print(y.value_counts())
    if i==0: print(y_test.value_counts())
        
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    
    # 2. CV to tune the parameter
    param_grid = {
                 'alpha':[3,5,7],
                 'solver': ['adam'],
                 'max_iter':[100, 120],
                 'momentum':[0.1, 0.5, 0.7],
                 'hidden_layer_sizes': [(70,)]} 
    
    
    CLF = MLPClassifier(activation="logistic",
                        validation_fraction=0,
                        random_state=42)
    
    gsearch = GridSearchCV(estimator = CLF, 
                           param_grid = param_grid, 
                           scoring=tuning_scoring, 
                           cv=cv)
    gsearch.fit(X,y)
    if LOG: print(gsearch.best_params_)
    
    # 3. Internal validation
    model = MLPClassifier(activation="logistic",validation_fraction=0,random_state=42,
                          solver=gsearch.best_params_['solver'],
                          alpha=gsearch.best_params_['alpha'],
                          max_iter=gsearch.best_params_['max_iter'],
                          momentum=gsearch.best_params_['momentum'],
                          hidden_layer_sizes=gsearch.best_params_['hidden_layer_sizes']
                         )
    
    scores = cross_validate(model, X, y, 
                            scoring=scoring, 
                            cv=cv, 
                            return_train_score= True)
    
    if LOG:
        print(i,'auc',scores['train_roc_auc'].mean(),scores['test_roc_auc'].mean())
        print(i,'ppv',scores['train_precision'].mean(),scores['test_precision'].mean())
        
    rets_internal['acc']  += list(scores['test_accuracy'])
    rets_internal['auc']  += list(scores['test_roc_auc'])
    rets_internal['prec']  += list(scores['test_precision'])
    rets_internal['recall']  += list(scores['test_recall'])
    rets_internal['sp']  += list(scores['test_specifty'])
    
    # 4. External validation
    model = MLPClassifier(activation="logistic",validation_fraction=0,random_state=42,
                          solver=gsearch.best_params_['solver'],
                          alpha=gsearch.best_params_['alpha'],
                          max_iter=gsearch.best_params_['max_iter'],
                          momentum=gsearch.best_params_['momentum'],
                          hidden_layer_sizes=gsearch.best_params_['hidden_layer_sizes']
                         )
    
    _, acc, auc, precision, recall, sp, tp, fp = baseline_fn(
        model, X, X_test, y, y_test, name = '')
    
    rets_cross['acc']  += [acc]
    rets_cross['auc']  += [auc]
    rets_cross['prec']  += [precision]
    rets_cross['recall']  += [recall]
    rets_cross['sp']  += [sp]
    
    if LOG:
        print(i,'auc',auc,'ppv',precision)

print()
ret = getmeanAndCI(rets_internal['auc'], "AUC")
ret = getmeanAndCI(rets_internal['prec'], "PPV")
ret = getmeanAndCI(rets_internal['recall'], "Sensitivity")
ret = getmeanAndCI(rets_internal['sp'], "Specificity")
ret = getmeanAndCI(rets_internal['acc'], "ACC")
print()
ret = getmeanAndCI(rets_cross['auc'], "AUC")
ret = getmeanAndCI(rets_cross['prec'], "PPV")
ret = getmeanAndCI(rets_cross['recall'], "Sensitivity")
ret = getmeanAndCI(rets_cross['sp'], "Specificity")
ret = getmeanAndCI(rets_cross['acc'], "ACC")

0    5
1    5
Name: Recurrence, dtype: int64
0    5
1    5
Name: Recurrence, dtype: int64
{'alpha': 5, 'hidden_layer_sizes': (70,), 'max_iter': 100, 'momentum': 0.1, 'solver': 'adam'}
0 auc 1.0 1.0
0 ppv 1.0 0.6
0 auc 0.4 ppv 0.5714285714285714
{'alpha': 5, 'hidden_layer_sizes': (70,), 'max_iter': 100, 'momentum': 0.1, 'solver': 'adam'}
10 auc 1.0 0.2
10 ppv 0.7009523809523809 0.4
10 auc 0.36 ppv 0.4444444444444444

AUC:0.48
 CI:0.38-0.58
PPV:0.455
 CI:0.41-0.5
Sensitivity:0.85
 CI:0.779-0.921
Specificity:0.09
 CI:0.033-0.147
ACC:0.47
 CI:0.428-0.512

AUC:0.412
 CI:0.345-0.479
PPV:0.533
 CI:0.505-0.562
Sensitivity:0.9
 CI:0.843-0.957
Specificity:0.21
 CI:0.146-0.274
ACC:0.555
 CI:0.508-0.602


In [25]:
if ALL:
    rets_internal_MLP = rets_internal
    rets_cross_MLP = rets_cross

if DEMOGRAPHCIS:
    rets_internal_MLP_demo = rets_internal
    rets_cross_MLP_demo = rets_cross

if MEDICAL:
    rets_internal_MLP_med = rets_internal 
    rets_cross_MLP_med = rets_cross

In [ ]:
_,p = st.ttest_ind(rets_internal_MLP['auc'], rets_cross_MLP['auc'])
print("auc %.3f" % p)
_,p = st.ttest_ind(rets_internal_MLP['prec'], rets_cross_MLP['prec'])
print("ppv %.3f" % p)
_,p = st.ttest_ind(rets_internal_MLP['recall'], rets_cross_MLP['recall'])
print("sn %.3f" % p)
_,p = st.ttest_ind(rets_internal_MLP['sp'], rets_cross_MLP['sp'])
print("sp %.3f" % p)
_,p = st.ttest_ind(rets_internal_MLP['acc'], rets_cross_MLP['acc'])
print("acc %.3f" % p)

# Random Forest

In [ ]:
rets_internal = {'acc':[], 'auc': [], 'prec': [], 'recall':[], 'sp': []}
rets_cross = {'acc':[], 'auc': [], 'prec': [], 'recall':[], 'sp': []}
for i in range(nTrials):
    if i%5 ==0: LOG=True
    else: LOG = False
        
    # 1. down sampling
    X, y = RandomUnderSampler().fit_resample(X_mgh, y_mgh)
    X_test, y_test = RandomUnderSampler().fit_resample(X_dfci, y_dfci)
    
    X = StandardScaler().fit_transform(X)
    X_test = StandardScaler().fit_transform(X_test)
    if i==0: print(y.value_counts())
    if i==0: print(y_test.value_counts())
    
    
    # 2. CV to tune the parameter
    param_grid = {'min_samples_split':range(5,10),
                  'n_estimators':range(50,91, 10)}
    CLF = RandomForestClassifier(random_state=42,)
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    gsearch = GridSearchCV(estimator = CLF,
                           param_grid = param_grid, 
                           scoring=tuning_scoring, 
                           cv=cv)
    gsearch.fit(X,y)
    if LOG: print(i,gsearch.best_params_,gsearch.best_score_)
    
    
    # internal
    model = RandomForestClassifier(random_state=42,
                                   n_estimators = gsearch.best_params_['n_estimators'],
                                   min_samples_split = gsearch.best_params_['min_samples_split'])
    
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    scores = cross_validate(model, X, y, 
                            scoring=scoring, 
                            cv=cv, return_train_score= True)
    if LOG:
        print('auc',scores['train_roc_auc'].mean(),scores['test_roc_auc'].mean())
        print('ppv',scores['train_precision'].mean(),scores['test_precision'].mean())
        
    rets_internal['acc']  += list(scores['test_accuracy'])
    rets_internal['auc']  += list(scores['test_roc_auc'])
    rets_internal['prec']  += list(scores['test_precision'])
    rets_internal['recall']  += list(scores['test_recall'])
    rets_internal['sp']  += list(scores['test_specifty'])
    
    # external
    model = RandomForestClassifier(random_state=42,
                                   n_estimators = gsearch.best_params_['n_estimators'],
                                   min_samples_split = gsearch.best_params_['min_samples_split'])
    
    _, acc, auc, precision, recall, sp, tp, fp = baseline_fn(
        model, X, X_test, y, y_test, name = '')
    
    rets_cross['acc']  += [acc]
    rets_cross['auc']  += [auc]
    rets_cross['prec']  += [precision]
    rets_cross['recall']  += [recall]
    rets_cross['sp']  += [sp]


print()
ret = getmeanAndCI(rets_internal['auc'], "AUC")
ret = getmeanAndCI(rets_internal['prec'], "PPV")
ret = getmeanAndCI(rets_internal['recall'], "Sensitivity")
ret = getmeanAndCI(rets_internal['sp'], "Specificity")
ret = getmeanAndCI(rets_internal['acc'], "ACC")
print()
ret = getmeanAndCI(rets_cross['auc'], "AUC")
ret = getmeanAndCI(rets_cross['prec'], "PPV")
ret = getmeanAndCI(rets_cross['recall'], "Sensitivity")
ret = getmeanAndCI(rets_cross['sp'], "Specificity")
ret = getmeanAndCI(rets_cross['acc'], "ACC")

In [ ]:
if ALL:
    rets_internal_RF = rets_internal
    rets_cross_RF = rets_cross

if DEMOGRAPHCIS:
    rets_internal_RF_demo = rets_internal
    rets_cross_RF_demo = rets_cross

if MEDICAL:
    rets_internal_RF_med = rets_internal 
    rets_cross_RF_med = rets_cross

In [ ]:
_,p = st.ttest_ind(rets_internal_RF['auc'], rets_cross_RF['auc'])
print("auc %.3f" % p)
_,p = st.ttest_ind(rets_internal_RF['prec'], rets_cross_RF['prec'])
print("ppv %.3f" % p)
_,p = st.ttest_ind(rets_internal_RF['recall'], rets_cross_RF['recall'])
print("sn %.3f" % p)
_,p = st.ttest_ind(rets_internal_RF['sp'], rets_cross_RF['sp'])
print("sp %.3f" % p)
_,p = st.ttest_ind(rets_internal_RF['acc'], rets_cross_RF['acc'])
print("acc %.3f" % p)

# SVM

In [ ]:
rets_internal = {'acc':[], 'auc': [], 'prec': [], 'recall':[], 'sp': []}
rets_cross = {'acc':[], 'auc': [], 'prec': [], 'recall':[], 'sp': []}
for i in range(nTrials):
    if i%5 == 0: LOG=True
    else: LOG = False
        
    # 1. down sampling
    X, y = RandomUnderSampler().fit_resample(X_mgh, y_mgh)
    X_test, y_test = RandomUnderSampler().fit_resample(X_dfci, y_dfci)
    if i==0: print(y.value_counts())
    if i==0: print(y_test.value_counts())
    
    X = StandardScaler().fit_transform(X)
    X_test = StandardScaler().fit_transform(X_test)
    
    
    # 2. CV to tune the parameter
    param_grid = {'C':[1, 1.5, 2,2.5]}
    CLF = SVC(kernel='rbf',probability=True)
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    gsearch = GridSearchCV(estimator = CLF, 
                           param_grid = param_grid, 
                           scoring=tuning_scoring,
                           cv=cv)
    gsearch.fit(X,y)
    if LOG: print(i,gsearch.best_params_)
    
    
    # internal
    model = SVC(kernel='rbf',probability=True,
               C = gsearch.best_params_['C']
               )
    
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    scores = cross_validate(model, X, y, 
                            scoring=scoring, 
                            cv=cv, 
                            return_train_score= True)
    rets_internal['acc']  += list(scores['test_accuracy'])
    rets_internal['auc']  += list(scores['test_roc_auc'])
    rets_internal['prec']  += list(scores['test_precision'])
    rets_internal['recall']  += list(scores['test_recall'])
    rets_internal['sp']  += list(scores['test_specifty'])
    
    if LOG:
        print(i, 'auc',scores['train_roc_auc'].mean(),scores['test_roc_auc'].mean())
        print(i, 'ppv',scores['train_precision'].mean(),scores['test_precision'].mean())
    
    # external
    model = SVC(kernel='rbf',probability=True,
               C = gsearch.best_params_['C']
               )
    
    _, acc, auc, precision, recall, sp, tp, fp = baseline_fn(
        model, X, X_test, y, y_test, name = '')
    
    rets_cross['acc']  += [acc]
    rets_cross['auc']  += [auc]
    rets_cross['prec']  += [precision]
    rets_cross['recall']  += [recall]
    rets_cross['sp']  += [sp]


print()
ret = getmeanAndCI(rets_internal['auc'], "AUC")
ret = getmeanAndCI(rets_internal['prec'], "PPV")
ret = getmeanAndCI(rets_internal['recall'], "Sensitivity")
ret = getmeanAndCI(rets_internal['sp'], "Specificity")
ret = getmeanAndCI(rets_internal['acc'], "ACC")
print()
ret = getmeanAndCI(rets_cross['auc'], "AUC")
ret = getmeanAndCI(rets_cross['prec'], "PPV")
ret = getmeanAndCI(rets_cross['recall'], "Sensitivity")
ret = getmeanAndCI(rets_cross['sp'], "Specificity")
ret = getmeanAndCI(rets_cross['acc'], "ACC")

In [ ]:

if ALL:
    rets_internal_SVM = rets_internal
    rets_cross_SVM = rets_cross

if DEMOGRAPHCIS:
    rets_internal_SVM_demo = rets_internal
    rets_cross_SVM_demo = rets_cross

if MEDICAL:
    rets_internal_SVM_med = rets_internal 
    rets_cross_SVM_med = rets_cross


In [ ]:
_,p = st.ttest_ind(rets_internal_SVM['auc'], rets_cross_SVM['auc'])
print("auc %.3f" % p)
_,p = st.ttest_ind(rets_internal_SVM['prec'], rets_cross_SVM['prec'])
print("ppv %.3f" % p)
_,p = st.ttest_ind(rets_internal_SVM['recall'], rets_cross_SVM['recall'])
print("sn %.3f" % p)
_,p = st.ttest_ind(rets_internal_SVM['sp'], rets_cross_SVM['sp'])
print("sp %.3f" % p)
_,p = st.ttest_ind(rets_internal_SVM['acc'], rets_cross_SVM['acc'])
print("acc %.3f" % p)

# Visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15,2), sharey=False, dpi=1024)

y = 0

# internal, LR
mu, ci = getmeanAndCI(rets_internal_LR['auc'], "LR", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax1.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_internal_LR['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='orange' )
ax2.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_internal_LR_demo['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax1.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_internal_LR_demo['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='gray' )
ax2.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_internal_LR_med['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax1.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_internal_LR_med['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='blue' )
ax2.plot(mu, y, 'o', color='green')

y += 1
# internal, MLP

mu, ci = getmeanAndCI(rets_internal_MLP_demo['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='gray', label="demographics only")
ax1.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_internal_MLP_demo['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax2.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_internal_MLP_med['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='blue', label="demographics + medical history")
ax1.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_internal_MLP_med['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax2.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_internal_MLP['auc'], "MLP", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='orange', label="all extracted features")
ax1.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_internal_MLP['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax2.plot(mu, y, 'o', color='#f44336')

y += 1
# internal, GB
mu, ci = getmeanAndCI(rets_internal_GB['auc'], "GB")
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax1.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_internal_GB['prec'], "")
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax2.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_internal_GB_demo['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax1.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_internal_GB_demo['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='gray' )
ax2.plot(mu, y, 'o', color='black')


mu, ci = getmeanAndCI(rets_internal_GB_med['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax1.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_internal_GB_med['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='blue' )
ax2.plot(mu, y, 'o', color='green')



y += 1
# internal, RF
#--all
mu, ci = getmeanAndCI(rets_internal_RF['auc'], "RF", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax1.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_internal_RF['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax2.plot(mu, y, 'o', color='#f44336')

#--demo
mu, ci = getmeanAndCI(rets_internal_RF_demo['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax1.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_internal_RF_demo['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax2.plot(mu, y, 'o', color='black')

#--demo
mu, ci = getmeanAndCI(rets_internal_RF_med['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax1.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_internal_RF_med['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax2.plot(mu, y, 'o', color='green')


y += 1
# internal, SVM
mu, ci = getmeanAndCI(rets_internal_SVM['auc'], "SVM", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax1.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_internal_SVM['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='orange' )
ax2.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_internal_SVM_demo['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax1.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_internal_SVM_demo['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='gray' )
ax2.plot(mu, y, 'o', color='black')


mu, ci = getmeanAndCI(rets_internal_SVM_med['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax1.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_internal_SVM_med['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='blue' )
ax2.plot(mu, y, 'o', color='green')



#---------------------------------------------------------------

ax1.legend(bbox_to_anchor=(0, 1.56), loc='upper left')

ax1.set_xlabel("AUC: mean and 95% confidence interval\n(internal validation)")
ax2.set_xlabel("PPV: mean and 95% confidence interval\n(internal validation)")


ax1.set_xlim(0.49, 0.88)
ax2.set_xlim(0.49, 0.84)

ax1.set_yticks([0, 1, 2, 3, 4])
ax1.set_yticklabels(["LR", "MLP", "GB", "RF", "SVM"]) 
ax2.set_yticks([0, 1, 2, 3, 4])
ax2.set_yticklabels(["LR", "MLP", "GB", "RF", "SVM"])  


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15,2), sharey=False, dpi=1024)

y = 0
# LR
mu, ci = getmeanAndCI(rets_cross_LR['auc'], "LR", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax1.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_cross_LR['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='orange' )
ax2.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_cross_LR_demo['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax1.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_cross_LR_demo['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='gray' )
ax2.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_cross_LR_med['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax1.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_cross_LR_med['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='blue' )
ax2.plot(mu, y, 'o', color='green')


y += 1
# MLP
mu, ci = getmeanAndCI(rets_cross_MLP_demo['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='gray', label="demographics only")
ax1.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_cross_MLP_demo['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax2.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_cross_MLP_med['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='blue', label="demographics + medical history")
ax1.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_cross_MLP_med['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax2.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_cross_MLP['auc'], "MLP", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='orange', label="all extracted features")
ax1.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_cross_MLP['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax2.plot(mu, y, 'o', color='#f44336')


y += 1
# GB
mu, ci = getmeanAndCI(rets_cross_GB['auc'], "GB")
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax1.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_cross_GB['prec'], "GB")
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='orange' )
ax2.plot(mu, y, 'o', color='#f44336')


mu, ci = getmeanAndCI(rets_cross_GB_demo['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax1.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_cross_GB_demo['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='gray' )
ax2.plot(mu, y, 'o', color='black')


mu, ci = getmeanAndCI(rets_cross_GB_med['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax1.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_cross_GB_med['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='blue' )
ax2.plot(mu, y, 'o', color='green')

y += 1
# RF
#--all
mu, ci = getmeanAndCI(rets_cross_RF['auc'], "RF", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax1.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_cross_RF['acc'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax2.plot(mu, y, 'o', color='#f44336')


#--demo
mu, ci = getmeanAndCI(rets_cross_RF_demo['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax1.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_cross_RF_demo['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax2.plot(mu, y, 'o', color='black')

#--demo
mu, ci = getmeanAndCI(rets_cross_RF_med['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax1.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_cross_RF_med['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax2.plot(mu, y, 'o', color='green')



y += 1
# internal, SVM
mu, ci = getmeanAndCI(rets_cross_SVM['auc'], "SVM", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='orange')
ax1.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_cross_SVM['acc'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='orange' )
ax2.plot(mu, y, 'o', color='#f44336')

mu, ci = getmeanAndCI(rets_cross_SVM_demo['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='gray')
ax1.plot(mu, y, 'o', color='black')

mu, ci = getmeanAndCI(rets_cross_SVM_demo['acc'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='gray' )
ax2.plot(mu, y, 'o', color='black')


mu, ci = getmeanAndCI(rets_cross_SVM_med['auc'], "", False)
ax1.plot((ci[0],ci[1]),(y,y),'ro--',color='blue')
ax1.plot(mu, y, 'o', color='green')

mu, ci = getmeanAndCI(rets_cross_SVM_med['prec'], "", False)
ax2.plot((ci[0],ci[1]),(y,y),'ro--',color='blue' )
ax2.plot(mu, y, 'o', color='green')



#---------------------------------------------------------------

# ax1.legend(bbox_to_anchor=(0, 1.56), loc='upper left')

ax1.set_xlabel("AUC: mean and 95% confidence interval\n(external validation)")
ax2.set_xlabel("PPV: mean and 95% confidence interval\n(external validation)")

ax1.set_xlim(0.49, 0.88)
ax2.set_xlim(0.49, 0.84)


ax1.set_yticks([0, 1, 2, 3, 4])
ax1.set_yticklabels(["LR", "MLP", "GB", "RF", "SVM"]) 
ax2.set_yticks([0, 1, 2, 3, 4])
ax2.set_yticklabels(["LR", "MLP", "GB", "RF", "SVM"])
